## Linear Probing exemple (squelette)

In [ ]:
import torch

# 1. Charger le modèle pré-entraîné
checkpoint = torch.load('checkpoint.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

# 2. Geler l'encodeur (backbone)
for param in model.parameters():
    param.requires_grad = False

# 3. Ajouter une couche linéaire pour la classification
class LinearProbingModel(nn.Module):
    def __init__(self, backbone, num_classes):
        super(LinearProbingModel, self).__init__()
        self.backbone = backbone
        self.fc = nn.Linear(backbone.output_dim, num_classes)  # Projection vers le nombre de classes

    def forward(self, x):
        with torch.no_grad():  # Ne met pas à jour les poids de l'encodeur
            features = self.backbone(x)
        return self.fc(features)


# 4. Entraîner la couche linéaire
model = LinearProbingModel(backbone=model.encoder_q, num_classes=1000)  # Exemple avec 1000 classes
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

# Boucle d'entraînement pour le linear probing
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

